In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
#         self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
#         h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
#         c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            
#             h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            
#             print("h_t2=",h_t2)
            output = self.linear(h_t)
            outputs += [output]
#             print(outputs)
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
#             h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [3]:

np.random.seed(2)

T = 2
L = 10
N = 4

x = np.empty((N, L), 'int64')
aa=np.array(range(L))
bb=np.random.randint(-4 * T, 4 * T, N)
cc=bb.reshape(N, 1)
x[:] = aa +cc 
print("aa=",aa)
print('bb=',bb)
print('cc=',cc)
x

aa= [0 1 2 3 4 5 6 7 8 9]
bb= [0 7 5 0]
cc= [[0]
 [7]
 [5]
 [0]]


array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9]], dtype=int64)

In [4]:

data = np.sin(x / 1.0 / T).astype('float64')
torch.save(data, open(r'D:\projects\pytorch-examples\time_sequence_prediction\traindata.pt', 'wb'))
data

array([[ 0.        ,  0.47942554,  0.84147098,  0.99749499,  0.90929743,
         0.59847214,  0.14112001, -0.35078323, -0.7568025 , -0.97753012],
       [-0.35078323, -0.7568025 , -0.97753012, -0.95892427, -0.70554033,
        -0.2794155 ,  0.21511999,  0.6569866 ,  0.93799998,  0.98935825],
       [ 0.59847214,  0.14112001, -0.35078323, -0.7568025 , -0.97753012,
        -0.95892427, -0.70554033, -0.2794155 ,  0.21511999,  0.6569866 ],
       [ 0.        ,  0.47942554,  0.84147098,  0.99749499,  0.90929743,
         0.59847214,  0.14112001, -0.35078323, -0.7568025 , -0.97753012]])

In [5]:
np.random.seed(0)
torch.manual_seed(0)
# load data and make training set
data = torch.load(r'D:\projects\pytorch-examples\time_sequence_prediction\traindata.pt')
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
print(input)
print(target)

tensor([[ 0.0000,  0.4794,  0.8415,  0.9975,  0.9093,  0.5985,  0.1411,
         -0.3508, -0.7568]], dtype=torch.float64)
tensor([[ 0.4794,  0.8415,  0.9975,  0.9093,  0.5985,  0.1411, -0.3508,
         -0.7568, -0.9775]], dtype=torch.float64)


In [12]:
from torch.autograd import Function,Variable

# Inherit from Function
class Linear(Function):

    # bias is an optional argument
    def forward(self, input, weight, bias=None):
        self.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = self.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if self.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if self.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and self.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

def linear(input, weight, bias=None):
    # First braces create a Function object. Any arguments given here
    # will be passed to __init__. Second braces will invoke the __call__
    # operator, that will then use forward() to compute the result and
    # return it.
    return Linear()(input, weight, bias)


class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter is a special kind of Variable, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters can never be volatile and, different than Variables,
        # they require gradients by default.
        self.weight = nn.Parameter(torch.Tensor(input_features, output_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return Linear()(input, self.weight, self.bias)
        #注意这个Linear是之前实现过的Linear



In [39]:
# import torch.nn.functional as F
def _pointwise_loss(lambd, lambd_optimized, input, target, size_average=True, reduce=True):
    if target.requires_grad:
        d = lambd(input, target)
        if not reduce:
            return d
        return torch.mean(d) if size_average else torch.sum(d)
    else:
        return lambd_optimized(input, target, size_average, reduce)
    
def mse_loss(input, target, size_average=True, reduce=True):
    r"""mse_loss(input, target, size_average=True, reduce=True) -> Tensor
    Measures the element-wise mean squared error.
    See :class:`~torch.nn.MSELoss` for details.
    """
    return _pointwise_loss(lambda a, b: (a - b) ** 2, torch._C._nn.mse_loss,
                           input, target, size_average, reduce)

class myMSELoss(nn.Module):
    def __init__(self, size_average=True, reduce=True):
        super(myMSELoss, self).__init__()
        self.size_average = size_average
        self.reduce = reduce

    def forward(self, input, target):
#         _assert_no_grad(target)
        return mse_loss(input, target, size_average=self.size_average, reduce=self.reduce)

out=Variable(torch.tensor(np.array([1,2])))
target=Variable(torch.tensor(np.array([3,4])))
criterion =myMSELoss()
loss = criterion(out, target)
loss.backward()

RuntimeError: mse_loss_forward is not implemented for type torch.IntTensor

In [45]:
from torch.autograd import Function,Variable

class FLinear(Function):
    def __init__(self, input, target):
        self.save_for_backward(input, target)        
#         return (input-target)**2
    
    def backward(self):
        input, target = self.saved_tensors       
        return 2*(input-target)

def linear(input, target):
    return FLinear()(input, target)


class Linear(nn.Module):
    def __init__(self):
        pass
    def forward(self, input,target):
        # See the autograd section for explanation of what happens here.
        return FLinear(input, target)
        #注意这个Linear是之前实现过的Linear

out=Variable(torch.tensor(np.array([1,2])))
target=Variable(torch.tensor(np.array([3,4])))
criterion =Linear()
loss = criterion.forward(out, target)
loss.backward()

ValueError: not enough values to unpack (expected 2, got 0)

In [28]:
def f():
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set
    data = torch.load(r'D:\projects\pytorch-examples\time_sequence_prediction\traindata.pt')
    input = torch.from_numpy(data[3:, :-1])
    target = torch.from_numpy(data[3:, 1:])
    test_input = torch.from_numpy(data[:3, :-1])
    test_target = torch.from_numpy(data[:3, 1:])
    # build the model
    seq = Sequence()
    seq.double()
#     criterion =nn.MSELoss()
    criterion =myMSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
    #begin to train
    for i in range(10):
        print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
#             print(input.size(0))
            out = seq(input)
            loss = criterion(out, target)
#             print('loss:', loss.item())
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict, no need to track gradient here
        with torch.no_grad():
            future = 100
            pred = seq(test_input, future=future)
            loss = criterion(pred[:, :-future], test_target)
#             print('test loss:', loss.item())
            y = pred.detach().numpy()
        # draw the result
        print("here1")
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
            plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
        draw(y[0], 'r')
#         draw(y[1], 'g')
#         draw(y[2], 'b')
        plt.savefig(r'D:\projects\pytorch-examples\time_sequence_prediction\predict%d.pdf'%i)
        plt.show()
        plt.close()
        
f()

STEP:  0


NameError: name '_pointwise_loss' is not defined

In [107]:
loss_fn = torch.nn.MSELoss(reduce=True, size_average=True)
input = torch.autograd.Variable(torch.randn(3,4))
target = torch.autograd.Variable(torch.randn(3,4))
loss = loss_fn(input, target)
print(input); print(target); print(loss)
print(input.size(), target.size(), loss.size())

tensor([[ 0.4520, -0.6208,  1.0974,  0.5748],
        [-0.8689, -0.4601, -2.0998, -0.1178],
        [-0.4503,  0.9647,  1.0128,  0.4169]])
tensor([[ 0.2566,  1.2857,  0.4050, -0.4893],
        [ 0.2160, -1.0855, -0.9608,  1.4725],
        [-0.3604, -1.3850, -0.9320,  0.7884]])
tensor(1.6774)
torch.Size([3, 4]) torch.Size([3, 4]) torch.Size([])
